# Spam Filter for Look2Social

## ~03/10

* Finished Collecting Data
* Communicated with Look2Social team: Figured out the goal of the program
* Communicated with instructors: Adjusted my approach to the project

## 03/14 ~ : Data Cleaning

### 03/14 - Overview of data
(screenshot of original data)
* 20,000 rows of data - not labeled for spam
* Six criteria for spam: 

    1) Marketing Focus (by social media marketing group)
    
    2) Bot Generated
    
    3) Known Spammer
    
    4) Corporate Posted
    
    5) Own Post
    
    6) Hijacking (just use tag, different content)


Bot Generated - can be labeled by either pattern of post ("just closed a deal...") and url

Corporate Posted - User input account name

Known spammer, Own Post - User input account name

Hijacking, Marketing - Need some eyeballing.

In [1]:
import pandas as pd
import numpy as np

### Suggestion from Miles

(pic of Miles)

"Use Machine Learning form early phase! Use it for labeling!" 

### Approach
1. create 7 columns for spam - one column per each criteria 
    one for final spam column that's default value is 0 but become 1 based on user declaration of spam

2. label bot generated and corporate generated first by their pattern

3. seperate out the rows with no label. (~7000 rows)

4. label first dozen~hundred for hijacking and marketing

5. use machine learning (Naive Bayes / Logistic Regression / Random Forest) to label, set up threshold high 

6. eyeball rest of 'em

### 03/17 - Steps before labeling data
Created new columns and index

In [2]:
df = pd.read_excel('data/data_prepping_zeros_index.xlsx')

In [7]:
df

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
0,0,just closed a deal in 27 hours using #AdobeSig...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Fe0YfarG31
1,1,just closed a deal in 2 days using #AdobeSign ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TTlCvaDE0V
2,2,just closed a deal in 2 hours using #AdobeSign...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/okmHDeM9bZ
3,3,just closed a deal in 26 hours using #AdobeSig...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
4,4,just closed a deal in 6 days using #AdobeSign ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Oi57q6jdXP
5,5,just closed a deal in 35 minutes using #AdobeS...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
6,6,just closed a deal in 3 days using #AdobeSign ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/su9NFDvRSm
7,7,just closed a deal in 58 minutes using #AdobeS...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Ir5AOw3Uv8
8,8,just closed a deal in 7 minutes using #AdobeSi...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TUP7YE2FjQ
9,9,So apparently IU is catching on and wonвЂ™t ju...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1. Corporate post

Criteria for corporate = client(DocuSign) and its competitors (OneSpan, SignNow, Adobe Sign)

Explored 'screen_name' column and came up with following list of corporate account name:

In [3]:
adobe = ['Acrobat','Acrobat_GU','adobe_mabhatia','adobe1234567','AdobeCare','AdobeDocCloud','AdobeExpCloud','AdobeGov','adobemax','AdobeNews','AdobePartner','adobesignstx','AdobeStarProps','AdobeUK']
onespan = ['atOneSpan','OneSpan','OneSpanSign']
docusign = ['DocuSign','DocuSignAPAC','DocuSignAPI','DocuSignIMPACT','DocuSignUK','DocuSignING']
signnow = ['signnow']

From this observation, I found that it would be possible to label for spam_corporate based on that whether screen name contains certain company name or not (would be more efficient)

* bonus found: Adobe leverages its branch companies
* Q: Do we need multiple accounts or one accoun for social media marketing?

In [4]:
combined_lst = adobe+onespan+docusign+signnow
#combined_lst

In [6]:
#df[df['spam_']]

In [7]:
#df.loc[df['screen_name'] in (combined_lst), 'spam_corporate'] = 1

In [8]:
#df.loc[df['c1'] == 'Value', 'c2'] = 10

In [5]:
df['spam_corporate'][df.screen_name.isin(combined_lst)] = 1

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


2. Bot generated post

Bot generated post for this case is by AdobeSign, in format of "Just closed a deal in..." / "Just signed an agreement in.." / "just connected my...". its expanded url column value is always adobe sign. 

For right now, we can use the assumption that if expanded url column value is link to AdobeSign, then it is bot generated post. However, it would be possible to process text and compare its vector (ex: use clustering) to capture the group of bot generated post with its distinct pattern (later task to elaborate the work)

Considering time restriction, I will proceed with this hypothesis.

In [6]:
df['spam_bot'][df.expanded_url == 'https://acrobat.adobe.com'] = 1

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
df.head()

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
0,0,just closed a deal in 27 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Fe0YfarG31
1,1,just closed a deal in 2 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TTlCvaDE0V
2,2,just closed a deal in 2 hours using #AdobeSign...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/okmHDeM9bZ
3,3,just closed a deal in 26 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
4,4,just closed a deal in 6 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Oi57q6jdXP


* Q: is bot generated post worth to do it? Is that why Adobe using it?

Known spammer, Own Post spam would be labeled with similar method like below:

In [13]:
df.columns

Index(['index', 'text', 'retweet_count', 'favorite_count', 'spam',
       'spam_marketing', 'spam_hijack', 'spam_corporate', 'spam_bot',
       'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'favorited', 'truncated', 'id_str',
       'in_reply_to_screen_name', 'source', 'retweeted', 'created_at',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'lang',
       'listed_count', 'verified', 'location', 'user_id_str', 'description',
       'geo_enabled', 'user_created_at', 'statuses_count', 'followers_count',
       'favourites_count', 'protected', 'user_url', 'name', 'time_zone',
       'user_lang', 'utc_offset', 'friends_count', 'screen_name',
       'country_code', 'country', 'place_type', 'full_name', 'place_name',
       'place_id', 'place_lat', 'place_lon', 'lat', 'lon', 'expanded_url',
       'url'],
      dtype='object')

And... finally eyeball time.
Let's seperate out the rows that none of spam categories value is 1.

In [14]:
df.loc[df['spam_bot'] == 1 | df['spam_corporate'] == 1]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [14]:
df[(df['spam_bot'] == 1) | (df['spam_corporate'] == 1)]

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
0,0,just closed a deal in 27 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Fe0YfarG31
1,1,just closed a deal in 2 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TTlCvaDE0V
2,2,just closed a deal in 2 hours using #AdobeSign...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/okmHDeM9bZ
3,3,just closed a deal in 26 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
4,4,just closed a deal in 6 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Oi57q6jdXP
5,5,just closed a deal in 35 minutes using #AdobeS...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
6,6,just closed a deal in 3 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/su9NFDvRSm
7,7,just closed a deal in 58 minutes using #AdobeS...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Ir5AOw3Uv8
8,8,just closed a deal in 7 minutes using #AdobeSi...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TUP7YE2FjQ
10,10,just closed a deal in 27 minutes using #AdobeS...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/PMDUd1jH0h


In [15]:
target_df = df[(df['spam_bot'] != 1) & (df['spam_corporate'] != 1)]
target_df

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
9,9,So apparently IU is catching on and wonвЂ™t ju...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,82,"The 11 Year Customer: Last week, I met a VP of...",NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://dlvr.it/QWJnCz,https://t.co/knHc6lFzj5
107,107,7 Reasons to Choose Adobe Sign for E-Signature...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://buff.ly/2LZaGMH,https://t.co/PrPAyMqtII
115,115,What is it with ink discrimination? I was just...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,129,"Explore Adobe Sign, a web base tool used for d...",NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cornell.sabacloud.com/Saba/Web_spf/NA1...,https://t.co/siOQk3vgHD
132,132,New version of Adobe Flash Player for Windows ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://happyasis.com/forums/topic/824/adobe-fl...,https://t.co/61jXV17sJg
143,143,"Adobe Fill &amp; Sign users, you can get aroun...",NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,151,Sign up for Coastline's Online DGA C164 Adobe ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.coastline.edu/,https://t.co/mMhtSaip96
163,163,Sapho Employee Experience Portal 4.9 is here! ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://bit.ly/2JjAcP0,https://t.co/dsTkn5EnBD
167,167,Sign up today for our summer 2018 classes! Top...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://bit.ly/2GqpOiR,https://t.co/GiMClaF3pR


In [18]:
13227+6784

20011

In [19]:
target_df = df[(df['spam_bot'] != 1) & (df['spam_corporate'] != 1)]

In [10]:
target_df.columns

Index(['index', 'text', 'retweet_count', 'favorite_count', 'spam',
       'spam_marketing', 'spam_hijack', 'spam_corporate', 'spam_bot',
       'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'favorited', 'truncated', 'id_str',
       'in_reply_to_screen_name', 'source', 'retweeted', 'created_at',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'lang',
       'listed_count', 'verified', 'location', 'user_id_str', 'description',
       'geo_enabled', 'user_created_at', 'statuses_count', 'followers_count',
       'favourites_count', 'protected', 'user_url', 'name', 'time_zone',
       'user_lang', 'utc_offset', 'friends_count', 'screen_name',
       'country_code', 'country', 'place_type', 'full_name', 'place_name',
       'place_id', 'place_lat', 'place_lon', 'lat', 'lon', 'expanded_url',
       'url'],
      dtype='object')

In [13]:
target_df2 = target_df[['index','text','spam_marketing','spam_hijack','description','screen_name','expanded_url']]

In [14]:
target_df2.head()

,index,text,spam_marketing,spam_hijack,description,screen_name,expanded_url
9,9,So apparently IU is catching on and wonвЂ™t ju...,0,0,BSU вЂ17 - SIU вЂ19 // Architecture & Interi...,cmdavis4cap,NaN
82,82,"The 11 Year Customer: Last week, I met a VP of...",0,0,Changing the way management consulting service...,MHPAdvisory,http://dlvr.it/QWJnCz
107,107,7 Reasons to Choose Adobe Sign for E-Signature...,0,0,Making your business STAND OUT in the digital ...,GleesonDigital,https://buff.ly/2LZaGMH
115,115,What is it with ink discrimination? I was just...,0,0,Mythical conservative academic. Don't tell any...,LadyGrammarian,NaN
129,129,"Explore Adobe Sign, a web base tool used for d...",0,0,Cornell Information Technologies. Get support ...,Cornell_IT,https://cornell.sabacloud.com/Saba/Web_spf/NA1...


In [1]:
#target_df2.to_excel("smalleyeball.xlsx", sheet_name='Sheet_name_1')

In [2]:
#target_df.to_excel("eyeball.xlsx", sheet_name='Sheet_name_1')

### 3/31 Finally done with labeling!

In [8]:
auto_labeled_df = df[(df['spam_bot'] == 1) | (df['spam_corporate'] == 1)]

In [9]:
hand_labeled_df = df[(df['spam_bot'] != 1) & (df['spam_corporate'] != 1)]

In [10]:
hand_labeled_df

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
9,9,So apparently IU is catching on and wonвЂ™t ju...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,82,"The 11 Year Customer: Last week, I met a VP of...",NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://dlvr.it/QWJnCz,https://t.co/knHc6lFzj5
107,107,7 Reasons to Choose Adobe Sign for E-Signature...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://buff.ly/2LZaGMH,https://t.co/PrPAyMqtII
115,115,What is it with ink discrimination? I was just...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,129,"Explore Adobe Sign, a web base tool used for d...",NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://cornell.sabacloud.com/Saba/Web_spf/NA1...,https://t.co/siOQk3vgHD
132,132,New version of Adobe Flash Player for Windows ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://happyasis.com/forums/topic/824/adobe-fl...,https://t.co/61jXV17sJg
143,143,"Adobe Fill &amp; Sign users, you can get aroun...",NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,151,Sign up for Coastline's Online DGA C164 Adobe ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.coastline.edu/,https://t.co/mMhtSaip96
163,163,Sapho Employee Experience Portal 4.9 is here! ...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://bit.ly/2JjAcP0,https://t.co/dsTkn5EnBD
167,167,Sign up today for our summer 2018 classes! Top...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://bit.ly/2GqpOiR,https://t.co/GiMClaF3pR


In [20]:
13227+6784 #confirming

20011

Combining two subsets of hand-labeled data

In [11]:
hand_partone = pd.read_excel('data/smalleyeball_partone.xlsx')
hand_parttwo = pd.read_excel('data/smalleyeball_parttwo.xlsx')

In [51]:
#hand_partone

In [52]:
#hand_parttwo

In [12]:
hand_combined = pd.concat([hand_partone,hand_parttwo])

In [53]:
#hand_combined

In [54]:
#hand_combined.columns

In [13]:
hand_combined.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1,inplace=True)

In [14]:
hand_combined.columns

Index(['index', 'text', 'spam_marketing', 'spam_hijack', 'description',
       'screen_name', 'expanded_url'],
      dtype='object')

In [55]:
#hand_combined

In [56]:
#hand_labeled_df

In [57]:
#print(hand_labeled_df.columns)

In [58]:
#print(hand_combined.columns)

array([0, 0, 1, ..., 1, 0, 0])

In [15]:
hand_labeled_df['spam_marketing'] = hand_combined['spam_marketing'].values

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
hand_labeled_df['spam_hijack'] = hand_combined['spam_hijack'].values

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [59]:
#hand_labeled_df

I don't have a confidence for consistency in data labeling & my understanding for the definition of spam marketing & hijack - it would be necessary for Look2Social to label once again with their standard.

It is clearly easier to label if you sort the file by post text - would be able to label repeating/similar post at once, which would save a lot of time.

There were so many insights from data labeling. I would personally recommend labeling data once a quarter / 2 quarters to understand the trend. It would capture some qualitative insights of the client(DocuSign) and its competitors (Adobe Sign, OneSpan, SignNow).

In [17]:
combined_labeled_df = pd.concat([hand_labeled_df,auto_labeled_df])

In [18]:
combined_labeled_df.sort_values(by='index', inplace=True)

In [19]:
combined_labeled_df

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
0,0,just closed a deal in 27 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Fe0YfarG31
1,1,just closed a deal in 2 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TTlCvaDE0V
2,2,just closed a deal in 2 hours using #AdobeSign...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/okmHDeM9bZ
3,3,just closed a deal in 26 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
4,4,just closed a deal in 6 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Oi57q6jdXP
5,5,just closed a deal in 35 minutes using #AdobeS...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
6,6,just closed a deal in 3 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/su9NFDvRSm
7,7,just closed a deal in 58 minutes using #AdobeS...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Ir5AOw3Uv8
8,8,just closed a deal in 7 minutes using #AdobeSi...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TUP7YE2FjQ
9,9,So apparently IU is catching on and wonвЂ™t ju...,NaN,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
combined_labeled_df.columns

Index(['index', 'text', 'retweet_count', 'favorite_count', 'spam',
       'spam_marketing', 'spam_hijack', 'spam_corporate', 'spam_bot',
       'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'favorited', 'truncated', 'id_str',
       'in_reply_to_screen_name', 'source', 'retweeted', 'created_at',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'lang',
       'listed_count', 'verified', 'location', 'user_id_str', 'description',
       'geo_enabled', 'user_created_at', 'statuses_count', 'followers_count',
       'favourites_count', 'protected', 'user_url', 'name', 'time_zone',
       'user_lang', 'utc_offset', 'friends_count', 'screen_name',
       'country_code', 'country', 'place_type', 'full_name', 'place_name',
       'place_id', 'place_lat', 'place_lon', 'lat', 'lon', 'expanded_url',
       'url'],
      dtype='object')

Let's cleaning data now. 

I will drop all column with only one unique value.

In [21]:
df2 = combined_labeled_df
for col in combined_labeled_df.columns:
    if col in ['spam', 'spam_own', 'spam_corporate', 'spam_known', 'spam_marketing', 'spam_hijack', 'spam_bot']:
        pass
    elif combined_labeled_df[col].nunique() == 1:
        df2 = df2.drop(col,axis=1)

In [22]:
df2.columns

Index(['index', 'text', 'retweet_count', 'favorite_count', 'spam',
       'spam_marketing', 'spam_hijack', 'spam_corporate', 'spam_bot',
       'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'id_str', 'in_reply_to_screen_name', 'source',
       'created_at', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str',
       'lang', 'listed_count', 'verified', 'location', 'user_id_str',
       'description', 'geo_enabled', 'user_created_at', 'statuses_count',
       'followers_count', 'favourites_count', 'user_url', 'name', 'time_zone',
       'user_lang', 'utc_offset', 'friends_count', 'screen_name',
       'country_code', 'country', 'place_type', 'full_name', 'place_name',
       'place_id', 'place_lat', 'place_lon', 'lat', 'lon', 'expanded_url',
       'url'],
      dtype='object')

In [23]:
df2.head()

,index,text,retweet_count,favorite_count,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,...,place_type,full_name,place_name,place_id,place_lat,place_lon,lat,lon,expanded_url,url
0,0,just closed a deal in 27 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Fe0YfarG31
1,1,just closed a deal in 2 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/TTlCvaDE0V
2,2,just closed a deal in 2 hours using #AdobeSign...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/okmHDeM9bZ
3,3,just closed a deal in 26 hours using #AdobeSig...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
4,4,just closed a deal in 6 days using #AdobeSign ...,NaN,NaN,0,0,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://acrobat.adobe.com,https://t.co/Oi57q6jdXP


Good number of columns are filled with NaN.

I personally think it is danger to fill the column if NaN is more than half of row. I will use that as a threshold

In [24]:
df2.dropna(thresh=(len(df)/2), axis=1, inplace=True)

In [25]:
df2.head()

,index,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,...,user_created_at,statuses_count,followers_count,favourites_count,name,user_lang,friends_count,screen_name,expanded_url,url
0,0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,...,Fri Sep 09 20:40:00 +0000 2011,6750,108,1,Joe Zarroli,en,12,IslandRealtyGrp,https://acrobat.adobe.com,https://t.co/Fe0YfarG31
1,1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,...,Fri May 25 01:27:15 +0000 2012,5018,47,2,Party Karacters,en,25,partykaracters,https://acrobat.adobe.com,https://t.co/TTlCvaDE0V
2,2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,...,Thu Dec 18 22:06:40 +0000 2008,29,60,10,annpodlozny,en,52,annpodlozny,https://acrobat.adobe.com,https://t.co/okmHDeM9bZ
3,3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,...,Wed Aug 08 20:11:12 +0000 2012,5332,66,0,Angela Dugan,en,182,AngelaRedondo1,https://acrobat.adobe.com,https://t.co/AAhUGDLZxH
4,4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,...,Thu Aug 26 20:50:06 +0000 2010,1337,225,19,Phebe,en,348,Makingmeevents,https://acrobat.adobe.com,https://t.co/Oi57q6jdXP


In [26]:
df2.columns

Index(['index', 'text', 'spam', 'spam_marketing', 'spam_hijack',
       'spam_corporate', 'spam_bot', 'spam_known', 'spam_own', 'Docusign',
       'onespan', 'signnow', 'adobe sign', 'id_str', 'source', 'created_at',
       'lang', 'listed_count', 'verified', 'location', 'user_id_str',
       'description', 'geo_enabled', 'user_created_at', 'statuses_count',
       'followers_count', 'favourites_count', 'name', 'user_lang',
       'friends_count', 'screen_name', 'expanded_url', 'url'],
      dtype='object')

In [32]:
df2[['id_str','source','created_at','lang','verified','user_id_str','geo_enabled','user_created_at']].head()

,id_str,source,created_at,lang,verified,user_id_str,geo_enabled,user_created_at
0,1004148045292889984,"<a href=""http://www.echosign.com/"" rel=""nofoll...",Tue Jun 05 23:48:44 +0000 2018,en,False,370896174,True,Fri Sep 09 20:40:00 +0000 2011
1,1005207775352380032,"<a href=""http://www.echosign.com/"" rel=""nofoll...",Fri Jun 08 21:59:44 +0000 2018,en,False,589580923,False,Fri May 25 01:27:15 +0000 2012
2,1005563350451929984,"<a href=""http://www.echosign.com/"" rel=""nofoll...",Sat Jun 09 21:32:39 +0000 2018,en,False,18226845,False,Thu Dec 18 22:06:40 +0000 2008
3,1004856511322119936,"<a href=""http://www.echosign.com/"" rel=""nofoll...",Thu Jun 07 22:43:56 +0000 2018,en,False,745859413,False,Wed Aug 08 20:11:12 +0000 2012
4,1005232370834979968,"<a href=""http://www.echosign.com/"" rel=""nofoll...",Fri Jun 08 23:37:28 +0000 2018,en,False,183367252,False,Thu Aug 26 20:50:06 +0000 2010


Following the advice from Matt Drury, the one of data scientists I admire most, I will start small - starting with 10 columns.

I want to start with

1) text: characteristic about post, will be TF-IDF

2) description: characteristic about account, will be TF-IDF

3) statuses/followers/favourites/friends/listed count: all measure the activity of the count (good numeric values as well - to identify account identity (too high vs. too low)

4) company boolean category (DocuSign, OneSpan, SignNow, AdobeSign): if certain company is likely to focus marketing / easy to highjack? 

5) created at: post created time would be a good predictor (if spam is likely to be posted on some time range)


Now, I will continue cleaning data.

I will remove rows with non-English and all remaining column I think which are irrelevant/less important for making a decision. 

In [34]:
df3 = df2[df2['lang']=='en']

In [35]:
df3['lang'].unique()

array(['en'], dtype=object)

In [36]:
df3.columns

Index(['index', 'text', 'spam', 'spam_marketing', 'spam_hijack',
       'spam_corporate', 'spam_bot', 'spam_known', 'spam_own', 'Docusign',
       'onespan', 'signnow', 'adobe sign', 'id_str', 'source', 'created_at',
       'lang', 'listed_count', 'verified', 'location', 'user_id_str',
       'description', 'geo_enabled', 'user_created_at', 'statuses_count',
       'followers_count', 'favourites_count', 'name', 'user_lang',
       'friends_count', 'screen_name', 'expanded_url', 'url'],
      dtype='object')

In [38]:
columns_to_drop = ['index','id_str','source','lang','verified',
                   'location','user_id_str','geo_enabled','user_created_at',
                   'name','user_lang','screen_name','expanded_url','url']

In [39]:
df3.drop(columns_to_drop, axis=1,inplace=True)

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [41]:
df3

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,signnow,adobe sign,created_at,listed_count,description,statuses_count,followers_count,favourites_count,friends_count
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,False,True,Tue Jun 05 23:48:44 +0000 2018,1,There's no substitute for experience! Call 609...,6750,108,1,12
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,Fri Jun 08 21:59:44 +0000 2018,14,We provide an affordable way to have a wonderf...,5018,47,2,25
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,False,True,Sat Jun 09 21:32:39 +0000 2018,0,NaN,29,60,10,52
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,False,True,Thu Jun 07 22:43:56 +0000 2018,2,NaN,5332,66,0,182
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,Fri Jun 08 23:37:28 +0000 2018,8,Carefully Crafted & Intentionally Inspired Eve...,1337,225,19,348
5,just closed a deal in 35 minutes using #AdobeS...,0,0,0,0,1,0,0,False,False,False,True,Tue Jun 05 22:10:18 +0000 2018,2,NaN,5332,66,0,182
6,just closed a deal in 3 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,Fri Jun 08 22:48:58 +0000 2018,5,SP Guard from Iconix uses enhancements to stan...,419,161,0,87
7,just closed a deal in 58 minutes using #AdobeS...,0,0,0,0,1,0,0,False,False,False,True,Fri Jun 08 22:09:47 +0000 2018,0,NaN,39,44,4,17
8,just closed a deal in 7 minutes using #AdobeSi...,0,0,0,0,1,0,0,False,False,False,True,Thu Jun 07 22:04:50 +0000 2018,1374,"Social Media Consultant, Real estate investor ...",335579,88063,14184,84426
9,So apparently IU is catching on and wonвЂ™t ju...,0,0,0,0,0,0,0,False,False,False,False,Sat Jun 09 21:10:37 +0000 2018,4,BSU вЂ17 - SIU вЂ19 // Architecture & Interi...,20653,339,15151,109


20006 rows (seems like 5 rows were written with foreign languages) and 19 columns.

Here, 7 columns are y-value (spam related columns)

I will manipulate 'created_at' column first. I will use (Matt's method) to create two additional columns from it.

I only need hour:minute:second, and convert it to decimal form.

In [49]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20006 entries, 0 to 20010
Data columns (total 19 columns):
text                20006 non-null object
spam                20006 non-null int64
spam_marketing      20006 non-null int64
spam_hijack         20006 non-null int64
spam_corporate      20006 non-null int64
spam_bot            20006 non-null int64
spam_known          20006 non-null int64
spam_own            20006 non-null int64
Docusign            20006 non-null bool
onespan             20006 non-null bool
signnow             20006 non-null bool
adobe sign          20006 non-null bool
created_at          20006 non-null object
listed_count        20006 non-null int64
description         14781 non-null object
statuses_count      20006 non-null int64
followers_count     20006 non-null int64
favourites_count    20006 non-null int64
friends_count       20006 non-null int64
dtypes: bool(4), int64(12), object(3)
memory usage: 2.5+ MB


In [78]:
df3['created_at'].head()

0    Tue Jun 05 23:48:44 +0000 2018
1    Fri Jun 08 21:59:44 +0000 2018
2    Sat Jun 09 21:32:39 +0000 2018
3    Thu Jun 07 22:43:56 +0000 2018
4    Fri Jun 08 23:37:28 +0000 2018
Name: created_at, dtype: object

take out hh:mm:ss, convert it to decimal number...

In [62]:
df3['time_created'] = pd.to_datetime(df3['created_at']).copy()

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
df3.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,signnow,adobe sign,created_at,listed_count,description,statuses_count,followers_count,favourites_count,friends_count,time_created
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,False,True,Tue Jun 05 23:48:44 +0000 2018,1,There's no substitute for experience! Call 609...,6750,108,1,12,2018-06-05 23:48:44+00:00
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,Fri Jun 08 21:59:44 +0000 2018,14,We provide an affordable way to have a wonderf...,5018,47,2,25,2018-06-08 21:59:44+00:00
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,False,True,Sat Jun 09 21:32:39 +0000 2018,0,NaN,29,60,10,52,2018-06-09 21:32:39+00:00
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,False,True,Thu Jun 07 22:43:56 +0000 2018,2,NaN,5332,66,0,182,2018-06-07 22:43:56+00:00
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,Fri Jun 08 23:37:28 +0000 2018,8,Carefully Crafted & Intentionally Inspired Eve...,1337,225,19,348,2018-06-08 23:37:28+00:00


In [79]:
df3['time_hms_only'] = df3['time_created'].dt.time
#unnecessary - I can work with time_created

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
df3['created_at_float'] = df3['time_created'].dt.hour + df3['time_created'].dt.minute/60 + df3['time_created'].dt.second/3600

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
df4 = df3.drop(['created_at','time_hms_only','time_created'],axis=1)

In [81]:
df4.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,signnow,adobe sign,listed_count,description,statuses_count,followers_count,favourites_count,friends_count,created_at_float
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,False,True,1,There's no substitute for experience! Call 609...,6750,108,1,12,23.812222
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,14,We provide an affordable way to have a wonderf...,5018,47,2,25,21.995556
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,False,True,0,NaN,29,60,10,52,21.544167
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,False,True,2,NaN,5332,66,0,182,22.732222
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,8,Carefully Crafted & Intentionally Inspired Eve...,1337,225,19,348,23.624444


Taking the advice from Jack (and Matt who told me the name of this method), I converted time with fourier transformation.

In [84]:
df4['time_float_sin'] = np.sin((df4['created_at_float']*2*np.pi)/24)

In [85]:
df4['time_float_cos'] = np.cos((df4['created_at_float']*2*np.pi)/24)

In [87]:
df4.drop('created_at_float',axis=1,inplace=True)

In [88]:
df4.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,signnow,adobe sign,listed_count,description,statuses_count,followers_count,favourites_count,friends_count,time_float_sin,time_float_cos
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,False,True,1,There's no substitute for experience! Call 609...,6750,108,1,12,-0.049140,0.998792
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,14,We provide an affordable way to have a wonderf...,5018,47,2,25,-0.501007,0.865443
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,False,True,0,NaN,29,60,10,52,-0.599548,0.800339
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,False,True,2,NaN,5332,66,0,182,-0.325843,0.945424
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,False,True,8,Carefully Crafted & Intentionally Inspired Eve...,1337,225,19,348,-0.098162,0.995170
